<a href="https://colab.research.google.com/github/Tymass/chess-player/blob/yolo-training/YOLO_chess_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [1]:
!pip install torch torchvision
!pip install ultralytics
!pip install -U --no-cache-dir gdown --pre
!pip install roboflow
!pip install pyyaml

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

## Imports

In [ ]:
from ultralytics import YOLO
import gdown
import os
import torch
import glob
from roboflow import Roboflow
import shutil
import yaml
import time
from google.colab import userdata
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## Functions definition

## Constants

In [ ]:
# Configuration parameters for machine learning model training.

EPOCHS = 600
# Specifies the total number of passes through the entire training dataset.
# A high number like 1000 indicates intensive training, aiming for deep model refinement, but requires monitoring to avoid overfitting.

PATIENCE = 20
# Determines the number of epochs with no improvement on a chosen metric before training is stopped early.
# This is a strategy to prevent overfitting by halting the training process if the model ceases to learn further.

BATCH_TRAINING = -1
# Specify batch size during YOLO training. -1 here means automatic way

LR0 = 0.0001
# The initial learning rate sets the step size at the start of training for adjusting the model weights.
# An appropriate value (neither too large nor too small) is crucial for effective training dynamics.

LRF = 0.2
# Final learning rate factor, used to adjust the learning rate over time.
# This parameter is part of a learning rate schedule that gradually reduces the learning rate, aiding in model convergence.

IMGSZ = 640
# The uniform size (height and width) to which all input images are resized before being fed into the model.
# Ensuring a consistent image size is essential for models to process batches of data efficiently.

CONF_THRESHOLD = 0.5
# Confidence threshold for predictions. Only predictions with a confidence score higher than this threshold will be considered.
# This helps in reducing false positives by filtering out predictions with low confidence.

IOU_THRESHOLD = 0.6
# Intersection Over Union (IOU) threshold for Non-Maximum Suppression (NMS).
# Determines how much overlap between bounding boxes is allowed. A higher threshold can reduce duplicate detections for the same object.

# Setting up device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Set the current working directory as the base directory.
#BASE_DIR = os.getcwd()

## Datasets downloads

In [ ]:
# In case we need to remove dataset folder
!rm -rf /content/datasets

We use [1](https://universe.roboflow.com/joseph-nelson/chess-pieces-new/dataset/24) dataset as baseline then fine tune model on our dataset [2](https://app.roboflow.com/tymek-byrwa-1p3fh/chesspiecesdetection-y9ljv/6).

Dont forget to get your own Roboflow API key.


In [ ]:
# Define the mode for dataset selection. Possible values: 'yolov9' or 'yolov8'.
mode = 'yolov8'
classes_nb = 12
API_key = userdata.get('API_key')

# We need to specify what dataset we use
fine_tunning = True

if fine_tunning:
  project_version = 6

  if mode == 'yolov9':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("tymek-byrwa-1p3fh").project("chesspiecesdetection-y9ljv")
    version = project.version(project_version)
    dataset = version.download("yolov9")
  elif mode == 'yolov8':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("tymek-byrwa-1p3fh").project("chesspiecesdetection-y9ljv")
    version = project.version(project_version)
    dataset = version.download("yolov8")
else:
  project_version = 24

  if mode == 'yolov8':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("joseph-nelson").project("chess-pieces-new")
    version = project.version(project_version)
    dataset = version.download("yolov8")
  elif mode == 'yolov8':
    rf = Roboflow(api_key=API_key)
    project = rf.workspace("joseph-nelson").project("chess-pieces-new")
    version = project.version(project_version)
    dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.28, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to chessPiecesDetection-6 in yolov8:: 100%|██████████| 672/672 [00:00<00:00, 6383.97it/s]


In [ ]:
def update_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)

    # Modify the specific fields
    data['train'] = '../train/images'
    data['val'] = '../valid/images'

    with open(file_path, 'w') as file:
        yaml.safe_dump(data, file)

In [ ]:
# We move downloaded dataset to 'dataset' floder and update yaml file

os.mkdir('/content/datasets')
source_dataset_path = os.getcwd() + '/' + dataset.name.replace(' ', '-') + '-' + project_version
source_dataset_path_copy = source_dataset_path.split('/')[1:]

source_dataset_path_copy.insert(-1, 'datasets')


dest_dataset_path = '/' + '/'.join(source_dataset_path_copy)
config_path = dest_dataset_path + '/data.yaml'
test_imgs_path = dest_dataset_path + '/test/images'

shutil.move(source_dataset_path, dest_dataset_path)

update_yaml(config_path)

## Initial train

In [ ]:
# Check the selected mode and initialize the corresponding YOLO model.
if mode == 'yolov9':
    model = YOLO('yolov9e.pt')  # Initialize YOLOv9 with the specified weights file.
elif mode == 'yolov8':
    model = YOLO('yolov8m.pt')  # Initialize YOLOv8 with the specified weights file.
else:
    print(f"Set up config var mode correctly.")  # Prompt to set the mode variable correctly if it's not 'yolov9' or 'yolov8'.

# Train the model with specified parameters.
model.train(data=config_path,   # Path to the YAML file with dataset paths.
            save=True,             # Enable saving the final model weights.
            epochs=EPOCHS,         # Number of epochs to train the model.
            device=DEVICE,          # Device setup
            patience=PATIENCE,     # Early stopping patience.
            batch=BATCH_TRAINING,  # Batch size for training.
            imgsz=IMGSZ,           # Input image size.
            lr0=LR0,               # Initial learning rate.
            lrf=LRF)               # Final learning rate.

# Evaluate the model and compute the validation metrics.
metrics = model.val()

# Save the computed metrics to a text file.
with open(f'{mode}_base_model_metrics.txt', 'w') as f:
    f.write(f"mAP 50-95: {metrics.box.map}\n")   # Write mAP at IoU=50:95.
    f.write(f"mAP 50: {metrics.box.map50}\n")    # Write mAP at IoU=50.
    f.write(f"mAP 75: {metrics.box.map75}\n")    # Write mAP at IoU=75.
    f.write(f"Category-wise mAP 50-95: {metrics.box.maps}\n")  # Write list of mAP50-95 for each category.


engine/trainer: task=detect, mode=train, model=best.pt, data=/content/datasets/chessPiecesDetection-6/data.yaml, epochs=600, time=None, patience=20, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, o

train: Scanning /content/datasets/chessPiecesDetection-6/train/labels... 270 images, 12 backgrounds, 0 corrupt: 100%|██████████| 270/270 [00:00<00:00, 1923.95it/s]

train: New cache created: /content/datasets/chessPiecesDetection-6/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/datasets/chessPiecesDetection-6/valid/labels... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<00:00, 472.09it/s]

val: New cache created: /content/datasets/chessPiecesDetection-6/valid/labels.cache


...


87 epochs completed in 0.296 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 52.0MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25846708 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.56it/s]


                   all         45        558      0.856      0.847      0.899      0.725
          bishop-black         30         39      0.766      0.769      0.843      0.641
          bishop-white         17         22      0.652      0.909      0.867      0.757
            king-black         27         27          1       0.88       0.93      0.702
            king-white         32         33      0.834      0.762      0.866      0.712
          knight-black         20         32      0.968      0.969      0.988      0.768
          knight-white         29         46      0.931      0.913      0.957       0.76
            pawn-black         32        114      0.941      0.834      0.956      0.737
            pawn-white         31        118          1      0.799      0.964      0.796
           queen-black         28         28      0.672      0.929      0.815      0.671
           queen-white         31         31      0.733          1      0.904      0.778
            rook-blac

val: Scanning /content/datasets/chessPiecesDetection-6/valid/labels.cache... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.49s/it]


                   all         45        558      0.843      0.858      0.899      0.725
          bishop-black         30         39      0.753      0.784      0.844      0.637
          bishop-white         17         22      0.631      0.909      0.867      0.754
            king-black         27         27      0.991      0.889       0.93      0.701
            king-white         32         33      0.798      0.788      0.866       0.71
          knight-black         20         32      0.955      0.969      0.988      0.764
          knight-white         29         46      0.942      0.913      0.958      0.761
            pawn-black         32        114      0.942      0.856      0.956      0.736
            pawn-white         31        118      0.993      0.814      0.966      0.804
           queen-black         28         28      0.646      0.929      0.815      0.672
           queen-white         31         31      0.717          1      0.904      0.778
            rook-blac

## Inference of initial trained model

In [ ]:
model.predict(source=test_imgs_path,
                conf=CONF_THRESHOLD,
                iou=IOU_THRESHOLD,
                imgsz=IMGSZ,
                device=DEVICE,
                save=True)


image 1/15 /content/datasets/chessPiecesDetection-6/test/images/1714384189053_jpg.rf.c62ea4d3eb886a39ec868ad23c98be65.jpg: 640x640 2 bishop-whites, 1 king-black, 1 king-white, 2 knight-whites, 6 pawn-blacks, 7 pawn-whites, 2 queen-blacks, 1 queen-white, 2 rook-blacks, 1 rook-white, 37.2ms
image 2/15 /content/datasets/chessPiecesDetection-6/test/images/1714384189080_jpg.rf.7550d439748821e919d0cbcaa7a5c754.jpg: 640x640 3 bishop-whites, 1 king-black, 1 king-white, 1 knight-black, 1 knight-white, 2 pawn-blacks, 1 pawn-white, 1 queen-white, 2 rook-blacks, 1 rook-white, 37.1ms
image 3/15 /content/datasets/chessPiecesDetection-6/test/images/1714384189179_jpg.rf.dd476d08691f6df709412926ea8aa738.jpg: 640x640 2 bishop-blacks, 4 bishop-whites, 2 knight-whites, 1 pawn-black, 1 queen-black, 1 queen-white, 1 rook-white, 34.2ms
image 4/15 /content/datasets/chessPiecesDetection-6/test/images/1714384189214_jpg.rf.a8f3a13abca056f0a2f271d963e599c9.jpg: 640x640 2 bishop-blacks, 1 bishop-white, 1 king-bla

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bishop-black', 1: 'bishop-white', 2: 'king-black', 3: 'king-white', 4: 'knight-black', 5: 'knight-white', 6: 'pawn-black', 7: 'pawn-white', 8: 'queen-black', 9: 'queen-white', 10: 'rook-black', 11: 'rook-white'}
 obb: None
 orig_img: array([[[255, 252, 247],
         [255, 252, 247],
         [255, 252, 247],
         ...,
         [  0,  32,  98],
         [  4,  33, 102],
         [ 26,  51, 123]],
 
        [[255, 252, 247],
         [255, 252, 247],
         [255, 252, 247],
         ...,
         [  0,  33,  99],
         [  3,  34, 103],
         [ 29,  58, 127]],
 
        [[255, 252, 247],
         [255, 252, 247],
         [255, 252, 247],
         ...,
         [  0,  30,  95],
         [  0,  28,  92],
         [ 24,  59, 123]],
 
        ...,
 
        [[234, 238, 233],
         [234, 238, 233],
         [234, 238, 233],
  

In [ ]:
# Calculate predictions time
def predictions_time(path):
    try:
        if not os.path.isdir(path):
            print("Podana ścieżka nie istnieje lub nie prowadzi do folderu.")
            return
        files = os.listdir(path)

        for file in files:
            t1 = time.time()

            results = model.predict(path + '/' + file)

            t2 = time.time()
            dt = t2 - t1

            print(f"Detection time: {dt:.4f} seconds")


    except Exception as e:
        print("Wystąpił błąd:", e)

In [ ]:
predictions_time('/content/datasets/chessPiecesDetection-6/test/images')


image 1/1 /content/datasets/chessPiecesDetection-6/test/images/1714384189268_jpg.rf.98657ecafdb118a5e600446967d05f62.jpg: 640x640 2 bishop-blacks, 1 bishop-white, 1 king-black, 1 king-white, 2 knight-blacks, 2 knight-whites, 1 pawn-black, 1 queen-black, 1 queen-white, 2 rook-blacks, 1 rook-white, 38.2ms
Speed: 6.2ms preprocess, 38.2ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)
Detection time: 0.0858 seconds

image 1/1 /content/datasets/chessPiecesDetection-6/test/images/1714384189053_jpg.rf.c62ea4d3eb886a39ec868ad23c98be65.jpg: 640x640 4 bishop-whites, 1 king-black, 1 king-white, 2 knight-whites, 8 pawn-blacks, 7 pawn-whites, 2 queen-blacks, 3 queen-whites, 2 rook-blacks, 1 rook-white, 42.7ms
Speed: 1.8ms preprocess, 42.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Detection time: 0.0793 seconds

image 1/1 /content/datasets/chessPiecesDetection-6/test/images/1714384189555_jpg.rf.ab20499b2fab4eb2cf3b2e37c7bbe5f5.jpg: 640x640 2 bishop-blacks,